In [1]:
import pandas as pd
import numpy as np

In [118]:
# Load in data
atp_ranks = pd.read_csv("data/atp_ranks.csv")
wta_ranks = pd.read_csv("data/wta_ranks.csv")
womens_stats = pd.read_csv("data/womens_df_prod.csv")
mens_stats = pd.read_csv("data/mens_df_prod.csv")

Merge rankings data

In [119]:
# Tied rankings
atp_ranks[~atp_ranks['rank'].str.isnumeric()]

,player,rank,week
903,Pablo Cuevas,904T,2023-06-26
904,Adam Neff,904T,2023-06-26
923,Menelaos Efstathiou,924T,2023-06-26
924,John McNally,924T,2023-06-26
1001,Lorenzo Gagliardo,1002T,2023-06-26
...,...,...,...
8143,Youssef Labbene,2057T,2023-08-22
8144,Ignacio Benzal Alia,2061T,2023-08-22
8145,Akram El Sallaly,2061T,2023-08-22
8146,Abhinav Sanjeev Shanmugam,2061T,2023-08-22


In [120]:
# Remove T from the rank
atp_ranks['rank_cleaned'] = pd.to_numeric(atp_ranks['rank'].str.replace('T', ''), errors='coerce')

In [121]:
atp_ranks['player_cleaned'] = atp_ranks['player'].apply(lambda word: " ".join(word.split()[1:])+", "+word.split()[0])
atp_ranks

,player,rank,week,rank_cleaned,player_cleaned
0,Carlos Alcaraz,1,2023-06-26,1,"Alcaraz, Carlos"
1,Novak Djokovic,2,2023-06-26,2,"Djokovic, Novak"
2,Daniil Medvedev,3,2023-06-26,3,"Medvedev, Daniil"
3,Casper Ruud,4,2023-06-26,4,"Ruud, Casper"
4,Stefanos Tsitsipas,5,2023-06-26,5,"Tsitsipas, Stefanos"
...,...,...,...,...,...
8145,Akram El Sallaly,2061T,2023-08-22,2061,"El Sallaly, Akram"
8146,Abhinav Sanjeev Shanmugam,2061T,2023-08-22,2061,"Sanjeev Shanmugam, Abhinav"
8147,Ioannis Xilas,2061T,2023-08-22,2061,"Xilas, Ioannis"
8148,Jesse Delaney,2065,2023-08-22,2065,"Delaney, Jesse"


In [122]:
wta_ranks

,player,rank,week
0,Swiatek Iga,1.0,2023-06-26
1,Sabalenka Aryna,2.0,2023-06-26
2,Rybakina Elena,3.0,2023-06-26
3,Pegula Jessica,4.0,2023-06-26
4,Garcia Caroline,5.0,2023-06-26
...,...,...,...
5511,Faltinger Ines,1390.0,2023-08-22
5512,Goldsmith Weinreich Paloma,1390.0,2023-08-22
5513,Ionescu Iulia Andreea,1390.0,2023-08-22
5514,Joint Maya,1390.0,2023-08-22


In [123]:
# Move the first name to the front
# This doesn't accont for people with two first names but I think those may be edge cases I don't need to worry about for now
def move_first(names):
    names = names.split()
    ret = names[0:-1]
    ret.insert(0, names[-1])
    return " ".join(ret)


wta_ranks['player_cleaned'] = wta_ranks['player'].apply(move_first)
wta_ranks['player_cleaned'] = wta_ranks['player_cleaned'].apply(lambda word: " ".join(word.split()[1:])+", "+word.split()[0])
wta_ranks


,player,rank,week,player_cleaned
0,Swiatek Iga,1.0,2023-06-26,"Swiatek, Iga"
1,Sabalenka Aryna,2.0,2023-06-26,"Sabalenka, Aryna"
2,Rybakina Elena,3.0,2023-06-26,"Rybakina, Elena"
3,Pegula Jessica,4.0,2023-06-26,"Pegula, Jessica"
4,Garcia Caroline,5.0,2023-06-26,"Garcia, Caroline"
...,...,...,...,...
5511,Faltinger Ines,1390.0,2023-08-22,"Faltinger, Ines"
5512,Goldsmith Weinreich Paloma,1390.0,2023-08-22,"Goldsmith Weinreich, Paloma"
5513,Ionescu Iulia Andreea,1390.0,2023-08-22,"Ionescu Iulia, Andreea"
5514,Joint Maya,1390.0,2023-08-22,"Joint, Maya"


In [97]:
atp_ranks.columns

Index(['player', 'rank', 'week', 'rank_cleaned', 'player_cleaned'], dtype='object')

In [124]:
wta_ranks = wta_ranks.drop(columns = ["player"])
atp_ranks = atp_ranks.drop(columns = ['player','rank'])
atp_ranks = atp_ranks.rename(columns ={"rank_cleaned":"rank"})

In [180]:
ranks = pd.concat([atp_ranks, wta_ranks])
ranks = ranks.rename(columns = {'week':'date','player_cleaned':'player'})
ranks

,date,rank,player
0,2023-06-26,1.0,"Alcaraz, Carlos"
1,2023-06-26,2.0,"Djokovic, Novak"
2,2023-06-26,3.0,"Medvedev, Daniil"
3,2023-06-26,4.0,"Ruud, Casper"
4,2023-06-26,5.0,"Tsitsipas, Stefanos"
...,...,...,...
5511,2023-08-22,1390.0,"Faltinger, Ines"
5512,2023-08-22,1390.0,"Goldsmith Weinreich, Paloma"
5513,2023-08-22,1390.0,"Ionescu Iulia, Andreea"
5514,2023-08-22,1390.0,"Joint, Maya"


In [181]:
ranks['player'] = ranks["player"].apply(lambda x: x.title())
ranks.to_csv("data/ranks.csv", index=False)

In [127]:
stats = pd.concat([womens_stats, mens_stats])
stats[["name_home","name_away"]] = stats[["name_home","name_away"]].apply(lambda column: column.apply(lambda x: x.title()))
stats

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,games_won,max_games_in_a_row,avg_game_length_set4,set4_length,avg_game_length_set5,set5_length,set4_diff,set4_games,set5_diff,set5_games
0,sr:sport_event:42023445,sr:competitor:66968,"Kudermetova, Veronika",12.0,14,13,8,9,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sr:sport_event:42023447,sr:competitor:41355,"Bogdan, Ana",NaN,6,12,7,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sr:sport_event:42023449,sr:competitor:256547,"Parks, Alycia",NaN,5,8,17,3,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sr:sport_event:42023459,sr:competitor:99131,"Bucsa, Cristina",NaN,6,34,15,10,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sr:sport_event:42023461,sr:competitor:44834,"Pegula, Jessica",4.0,3,20,14,11,5,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,sr:sport_event:43216857,sr:competitor:163504,"Medvedev, Daniil",3.0,8,10,11,7,9,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,sr:sport_event:43216863,sr:competitor:407573,"Alcaraz, Carlos",1.0,3,5,10,12,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
492,sr:sport_event:43216849,sr:competitor:808628,"Shelton, Ben",NaN,5,20,11,4,2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,sr:sport_event:43216855,sr:competitor:407573,"Alcaraz, Carlos",1.0,0,25,14,12,1,0,...,NaN,NaN,6.661111,59.95,NaN,NaN,-3.0,9.0,NaN,NaN


In [128]:
stats_c = stats[["date","name_home", "seed_home", "name_away", "seed_away","men"]]
stats_c

,date,name_home,seed_home,name_away,seed_away,men
0,2023-06-26,"Kudermetova, Veronika",12.0,"Kanepi, Kaia",NaN,False
1,2023-06-26,"Bogdan, Ana",NaN,"Samsonova, Liudmila",15.0,False
2,2023-06-26,"Parks, Alycia",NaN,"Friedsam, Anna-Lena",NaN,False
3,2023-06-26,"Bucsa, Cristina",NaN,"Rakhimova, Kamilla",NaN,False
4,2023-06-26,"Pegula, Jessica",4.0,"Davis, Lauren",NaN,False
...,...,...,...,...,...,...
490,2023-08-22,"Medvedev, Daniil",3.0,"Rublev, Andrey",8.0,True
491,2023-08-22,"Alcaraz, Carlos",1.0,"Zverev, Alexander",12.0,True
492,2023-08-22,"Shelton, Ben",NaN,"Djokovic, Novak",2.0,True
493,2023-08-22,"Alcaraz, Carlos",1.0,"Medvedev, Daniil",3.0,True


In [129]:
print("stats_c")

for col in stats_c.columns:
    if any(stats_c[col].isna()):
        print(col)

print("ranks")
for col in ranks.columns:
    if any(ranks[col].isna()):
        print(col)

stats_c
seed_home
seed_away
ranks


Merging ranks and stats

In [130]:
import regex as re
from functools import reduce

In [312]:
# merge stats and ranks (only for home)
merged = stats_c.merge(ranks, how = "left", left_on=["date","name_home"], right_on=["date","player"]).rename(columns={"rank":"rank_home","player":"player_home"})

In [313]:
# Find missing players for home
missing_players = []
found_players = []
to_fix = []
for row in merged[(merged['rank_home'].isna())].iterrows():
    name = row[1].name_home
    name_list = re.split(", | ",name)
    potential_ppl = []
    for part in name_list:
        exists = ranks['player'].apply(lambda x: bool(re.search(part, x, flags=re.IGNORECASE)))
        potential_ppl.append(ranks[exists])
    
    found = reduce(lambda x, y: pd.merge(x, y, on=['date','rank','player']), potential_ppl)
    if found.empty:
        # Try again only search for last name
        exists = ranks['player'].apply(lambda x: bool(re.search(name_list[0], x, flags=re.IGNORECASE)))
        found = ranks[exists]
        if len(found)==4:
            if name not in found_players:
                print(f"FOUND: {name}")
                corrected_values = found.copy()
                corrected_values.loc[:, 'player_corrected'] = name
                to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
                found_players.append(name)
        else:
            if name not in missing_players:
                print(f"LOST: {name}")
                missing_players.append(name)
        
    else:
        if name not in found_players:
            print(f"FOUND: {name}")
            corrected_values = found.copy()
            corrected_values.loc[:,'player_corrected'] = name
            to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
            found_players.append(name)

# # represents a new dataframe of the remaining ranked players that were missed in the previous iteration
missed_ranks = pd.concat(to_fix)
missed_ranks = missed_ranks.drop_duplicates()

merged2 = merged.merge(missed_ranks, how = "left", left_on=["date","name_home"], right_on=["date","player_corrected"])#.drop(columns=["player_x","player_y"])
merged2['name_home'] = merged2.apply(lambda row: row['name_home'] if pd.isna(row['player_corrected']) else row['player_corrected'], axis=1)
merged2['rank_home'] = merged2.apply(lambda row: row['rank_home'] if pd.isna(row['rank']) else row['rank'], axis=1)

FOUND: Yuan, Yue
FOUND: Tsurenko, Lesia
FOUND: Burrage, Jodie Anna
FOUND: Kasatkina, Daria
FOUND: Cirstea, Sorana
LOST: Cristian, Jaqueline Adina
FOUND: Stephens, Sloane
FOUND: Minnen, Greet
LOST: Riske-Amritraj, Alison
FOUND: Begu, Irina-Camelia
LOST: Fernandez, Leylah Annie
FOUND: Andreescu, Bianca
FOUND: Azarenka, Victoria
FOUND: Parrizas Diaz, Nuria
FOUND: Samsonova, Liudmila
LOST: Maria, Tatjana
FOUND: Schmiedlova, Anna-Karolina
FOUND: Krunic, Aleksandra
FOUND: Gauff, Coco
FOUND: Miyazaki, Yuriko Lily
FOUND: Tig, Patricia Maria
LOST: Wolf, Jeffrey John
FOUND: Etcheverry, Tomas Martin
FOUND: Galan, Daniel Elahi
FOUND: Barrios Vera, Marcelo Tomas
FOUND: Tirante, Thiago Agustin
FOUND: Olivieri, Genaro Alberto
FOUND: Varillas, Juan Pablo
FOUND: Cerundolo, Juan Manuel
FOUND: Hsu, Yu Hsiou


In [314]:
merged2 = merged2.drop(columns = ["rank","player","player_corrected","player_home"])
merged2

,date,name_home,seed_home,name_away,seed_away,men,rank_home
0,2023-06-26,"Kudermetova, Veronika",12.0,"Kanepi, Kaia",NaN,False,12.0
1,2023-06-26,"Bogdan, Ana",NaN,"Samsonova, Liudmila",15.0,False,61.0
2,2023-06-26,"Parks, Alycia",NaN,"Friedsam, Anna-Lena",NaN,False,47.0
3,2023-06-26,"Bucsa, Cristina",NaN,"Rakhimova, Kamilla",NaN,False,77.0
4,2023-06-26,"Pegula, Jessica",4.0,"Davis, Lauren",NaN,False,4.0
...,...,...,...,...,...,...,...
983,2023-08-22,"Medvedev, Daniil",3.0,"Rublev, Andrey",8.0,True,3.0
984,2023-08-22,"Alcaraz, Carlos",1.0,"Zverev, Alexander",12.0,True,1.0
985,2023-08-22,"Shelton, Ben",NaN,"Djokovic, Novak",2.0,True,46.0
986,2023-08-22,"Alcaraz, Carlos",1.0,"Medvedev, Daniil",3.0,True,1.0


In [315]:
# # Do the same thing for away
merged = merged2.merge(ranks, how = "left", left_on=["date","name_away"], right_on=["date","player"]).rename(columns={"rank":"rank_away","player":"player_away"})

# Find missing players for home
found_players = []
to_fix = []
for row in merged[(merged['rank_away'].isna())].iterrows():
    name = row[1].name_away
    name_list = re.split(", | ",name)
    potential_ppl = []
    for part in name_list:
        exists = ranks['player'].apply(lambda x: bool(re.search(part, x, flags=re.IGNORECASE)))
        potential_ppl.append(ranks[exists])
    
    found = reduce(lambda x, y: pd.merge(x, y, on=['date','rank','player']), potential_ppl)
    if found.empty:
        # Try again only search for last name
        exists = ranks['player'].apply(lambda x: bool(re.search(name_list[0], x, flags=re.IGNORECASE)))
        found = ranks[exists]
        if len(found)==4:
            if name not in found_players:
                print(f"FOUND: {name}")
                corrected_values = found.copy()
                corrected_values.loc[:, 'player_corrected'] = name
                to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
                found_players.append(name)
        else:
            if name not in missing_players:
                print(f"LOST: {name}")
                missing_players.append(name)
        
    else:
        if name not in found_players:
            print(f"FOUND: {name}")
            corrected_values = found.copy()
            corrected_values.loc[:,'player_corrected'] = name
            to_fix.append(ranks.merge(corrected_values, on=["date", 'rank','player']))
            found_players.append(name)

missed_ranks = pd.concat(to_fix)
missed_ranks = missed_ranks.drop_duplicates()

merged2 = merged.merge(missed_ranks, how = "left", left_on=["date","name_away"], right_on=["date","player_corrected"])
merged2['name_away'] = merged2.apply(lambda row: row['name_away'] if pd.isna(row['player_corrected']) else row['player_corrected'], axis=1)
merged2['rank_away'] = merged2.apply(lambda row: row['rank_away'] if pd.isna(row['rank']) else row['rank'], axis=1)

FOUND: Samsonova, Liudmila
FOUND: Azarenka, Victoria
FOUND: Teichmann, Jil
LOST: Mcnally, Catherine
FOUND: Gauff, Coco
FOUND: Burrage, Jodie Anna
FOUND: Schmiedlova, Anna-Karolina
FOUND: Parrizas Diaz, Nuria
FOUND: Tsurenko, Lesia
FOUND: Andreescu, Bianca
FOUND: Cirstea, Sorana
FOUND: Begu, Irina-Camelia
FOUND: Kasatkina, Daria
FOUND: Stephens, Sloane
FOUND: Starodubtseva, Yuliia
FOUND: Osaka, Naomi
FOUND: Kerber, Angelique
FOUND: Yuan, Yue
FOUND: Ruse, Elena-Gabriela
FOUND: Minnen, Greet
FOUND: Varillas, Juan Pablo
FOUND: Barrios Vera, Marcelo Tomas
FOUND: Galan, Daniel Elahi
FOUND: Cerundolo, Juan Manuel
FOUND: Tirante, Thiago Agustin
FOUND: Olivieri, Genaro Alberto
FOUND: Etcheverry, Tomas Martin
FOUND: Balazs, Attila
FOUND: Hsu, Yu Hsiou


In [316]:
# Only a few players could not be merged, we can just drop these or have a NA seed and rank
print(missing_players)

['Cristian, Jaqueline Adina', 'Riske-Amritraj, Alison', 'Fernandez, Leylah Annie', 'Maria, Tatjana', 'Wolf, Jeffrey John', 'Mcnally, Catherine']


In [319]:
merged = merged2.drop(columns = ["rank","player","player_corrected","player_away"])
merged

,date,name_home,seed_home,name_away,seed_away,men,rank_home,rank_away
0,2023-06-26,"Kudermetova, Veronika",12.0,"Kanepi, Kaia",NaN,False,12.0,101.0
1,2023-06-26,"Bogdan, Ana",NaN,"Samsonova, Liudmila",15.0,False,61.0,15.0
2,2023-06-26,"Parks, Alycia",NaN,"Friedsam, Anna-Lena",NaN,False,47.0,87.0
3,2023-06-26,"Bucsa, Cristina",NaN,"Rakhimova, Kamilla",NaN,False,77.0,70.0
4,2023-06-26,"Pegula, Jessica",4.0,"Davis, Lauren",NaN,False,4.0,44.0
...,...,...,...,...,...,...,...,...
983,2023-08-22,"Medvedev, Daniil",3.0,"Rublev, Andrey",8.0,True,3.0,8.0
984,2023-08-22,"Alcaraz, Carlos",1.0,"Zverev, Alexander",12.0,True,1.0,12.0
985,2023-08-22,"Shelton, Ben",NaN,"Djokovic, Novak",2.0,True,46.0,2.0
986,2023-08-22,"Alcaraz, Carlos",1.0,"Medvedev, Daniil",3.0,True,1.0,3.0


In [320]:
merged['rs_home'] = merged['seed_home'].fillna(merged['rank_home'])
merged['rs_away'] = merged['seed_away'].fillna(merged['rank_away'])
merged

,date,name_home,seed_home,name_away,seed_away,men,rank_home,rank_away,rs_home,rs_away
0,2023-06-26,"Kudermetova, Veronika",12.0,"Kanepi, Kaia",NaN,False,12.0,101.0,12.0,101.0
1,2023-06-26,"Bogdan, Ana",NaN,"Samsonova, Liudmila",15.0,False,61.0,15.0,61.0,15.0
2,2023-06-26,"Parks, Alycia",NaN,"Friedsam, Anna-Lena",NaN,False,47.0,87.0,47.0,87.0
3,2023-06-26,"Bucsa, Cristina",NaN,"Rakhimova, Kamilla",NaN,False,77.0,70.0,77.0,70.0
4,2023-06-26,"Pegula, Jessica",4.0,"Davis, Lauren",NaN,False,4.0,44.0,4.0,44.0
...,...,...,...,...,...,...,...,...,...,...
983,2023-08-22,"Medvedev, Daniil",3.0,"Rublev, Andrey",8.0,True,3.0,8.0,3.0,8.0
984,2023-08-22,"Alcaraz, Carlos",1.0,"Zverev, Alexander",12.0,True,1.0,12.0,1.0,12.0
985,2023-08-22,"Shelton, Ben",NaN,"Djokovic, Novak",2.0,True,46.0,2.0,46.0,2.0
986,2023-08-22,"Alcaraz, Carlos",1.0,"Medvedev, Daniil",3.0,True,1.0,3.0,1.0,3.0


,date,name_home,seed_home,name_away,seed_away,men,rank_home,rank_away,rs_home,rs_away
39,2023-06-26,"Cristian, Jaqueline Adina",NaN,"Bronzetti, Lucia",NaN,False,NaN,65.0,NaN,65.0
53,2023-06-26,"Riske-Amritraj, Alison",NaN,"Badosa, Paula",NaN,False,NaN,34.0,NaN,34.0
77,2023-06-26,"Fernandez, Leylah Annie",NaN,"Garcia, Caroline",5.0,False,NaN,5.0,NaN,5.0
161,2023-05-22,"Maria, Tatjana",NaN,"Haddad Maia, Beatriz",14.0,False,NaN,14.0,NaN,14.0
198,2023-05-22,"Fernandez, Leylah Annie",NaN,"Tauson, Clara",NaN,False,NaN,126.0,NaN,126.0
269,2024-01-07,"Maria, Tatjana",NaN,"Osorio, Camila",NaN,False,NaN,81.0,NaN,81.0
276,2024-01-07,"Cristian, Jaqueline Adina",NaN,"Siniakova, Katerina",NaN,False,NaN,50.0,NaN,50.0
325,2024-01-07,"Maria, Tatjana",NaN,"Paolini, Jasmine",26.0,False,NaN,29.0,NaN,26.0
407,2023-08-22,"Maria, Tatjana",NaN,"Martic, Petra",NaN,False,NaN,36.0,NaN,36.0
505,2023-06-26,"Wolf, Jeffrey John",NaN,"Couacaud, Enzo",NaN,True,NaN,158.0,NaN,158.0


In [80]:
womens = pd.merge(womens_stats, ranks, left_on=['name_home', 'date'], right_on=['player', 'date'], how='left').rename(columns={'rank':'rank_home'}).drop(['player','date'], axis=1).merge(ranks, left_on=['name_away', 'date'], right_on=['player', 'date'], how='left').rename(columns={'rank':'rank_away'}).drop(['player'], axis=1)

mens = pd.merge(mens_stats, ranks, left_on=['name_home', 'date'], right_on=['player', 'date'], how='left').rename(columns={'rank':'rank_home'}).drop(['player','date'], axis=1).merge(ranks, left_on=['name_away', 'date'], right_on=['player', 'date'], how='left').rename(columns={'rank':'rank_away'}).drop(['player','date'], axis=1)

# This might not always work ^^^ check !

w = womens[["date",'name_home','seed_home','name_away','seed_away']]
m = mens[["date",'name_home','seed_home','name_away','seed_away']]


KeyError: 'date'

In [29]:
ranks[ranks['player_cleaned'].str.contains("Samsonova")]

,week,rank,player_cleaned
14,2023-06-26,15.0,"Samsonova, Ludmilla"
1387,2023-05-22,15.0,"Samsonova, Ludmilla"
2738,2024-01-07,15.0,"Samsonova, Ludmilla"
4129,2023-08-22,14.0,"Samsonova, Ludmilla"


In [35]:
womens['rs_home'] = womens['seed_home'].fillna(womens['rank_home'])
womens['rs_away'] = womens['seed_away'].fillna(womens['rank_away'])

In [75]:
missing_w = pd.concat([w[w['rs_home'].isna()],w[w['rs_away'].isna()]])

KeyError: 'rs_home'

In [71]:
missing_men = pd.concat([mens[mens['rs_home'].isna()],mens[mens['rs_away'].isna()]])
missing_men

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,set3_diff,set3_games,set4_diff,set4_games,set5_diff,set5_games,rank_home,rank_away,rs_home,rs_away
3,sr:sport_event:42023963,sr:competitor:658475,"van Assche, Luca",NaN,6,21,15,4,2,6,...,-4,8,-2.0,10.0,NaN,NaN,NaN,47.0,NaN,47.0
12,sr:sport_event:42023955,sr:competitor:234048,"Wolf, Jeffrey John",NaN,11,14,7,7,2,4,...,1,13,NaN,NaN,NaN,NaN,NaN,158.0,NaN,158.0
75,sr:sport_event:42001163,sr:competitor:92074,"Galan, Daniel Elahi",NaN,12,28,5,11,3,2,...,3,9,1.0,13.0,NaN,NaN,NaN,230.0,NaN,230.0
78,sr:sport_event:42001153,sr:competitor:132834,"Barrios Vera, Marcelo Tomas",NaN,5,21,17,12,4,3,...,-4,8,-6.0,6.0,NaN,NaN,NaN,123.0,NaN,123.0
98,sr:sport_event:42001047,sr:competitor:92074,"Galan, Daniel Elahi",NaN,9,15,36,12,10,1,...,1,13,-3.0,9.0,5.0,7.0,NaN,61.0,NaN,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,sr:sport_event:43216963,sr:competitor:131442,"Mmoh, Michael",NaN,10,19,19,5,2,7,...,3,9,-3.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN
479,sr:sport_event:43216967,sr:competitor:163504,"Medvedev, Daniil",3.0,11,7,12,8,6,7,...,1,13,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN
480,sr:sport_event:43216865,sr:competitor:138546,"Paul, Tommy",14.0,6,26,7,5,3,8,...,2,10,-2.0,10.0,NaN,NaN,NaN,NaN,14.0,NaN
484,sr:sport_event:43216943,sr:competitor:407573,"Alcaraz, Carlos",1.0,2,6,12,5,5,3,...,2,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [41]:
missing_women = list(pd.unique(womens[womens['rs_home'].isna()].name_home))+list((pd.unique(womens[womens['rs_away'].isna()].name_away)))
pd.unique(missing_women)

/var/folders/7_/mtdhgw6s63dc18509m14sztm0000gn/T/ipykernel_62551/704642689.py:2: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.unique(missing_women)


array(['Yuan, Yue', 'Tsurenko, Lesia', 'Burrage, Jodie Anna',
       'Cirstea, Sorana', 'Cristian, Jaqueline Adina', 'Stephens, Sloane',
       'Minnen, Greet', 'Riske-Amritraj, Alison',
       'Fernandez, Leylah Annie', 'Andreescu, Bianca',
       'Parrizas Diaz, Nuria', 'Maria, Tatjana',
       'Schmiedlova, Anna-Karolina', 'Krunic, Aleksandra',
       'Kessler, McCartney', 'Miyazaki, Yuriko Lily',
       'Begu, Irina-Camelia', 'Tig, Patricia Maria', 'Teichmann, Jil',
       'McNally, Catherine', 'Starodubtseva, Yuliia', 'Osaka, Naomi',
       'Kerber, Angelique', 'Ruse, Elena-Gabriela'], dtype=object)

In [45]:
ranks[ranks['player_cleaned']=='Murray, Andy']

,week,rank,player_cleaned
38,2023-06-26,39.0,"Murray, Andy"
2057,2023-05-22,41.0,"Murray, Andy"
4068,2024-01-01,42.0,"Murray, Andy"
6120,2023-08-21,37.0,"Murray, Andy"


In [46]:
mens[mens['name_home']=='Murray, Andy']

,event_id,player_id_home,name_home,seed_home,aces_home,backhand_errors_home,backhand_unforced_errors_home,backhand_winners_home,breakpoints_won_home,double_faults_home,...,set3_diff,set3_games,set4_diff,set4_games,set5_diff,set5_games,rank_home,rank_away,rs_home,rs_away
87,sr:sport_event:42001103,sr:competitor:15126,"Murray, Andy",NaN,13,23,12,15,1,2,...,2,10,-1.0,13.0,-2.0,10.0,39.0,5.0,39.0,5.0
284,sr:sport_event:46881051,sr:competitor:15126,"Murray, Andy",NaN,6,6,17,4,1,2,...,-4,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0
420,sr:sport_event:43232741,sr:competitor:15126,"Murray, Andy",NaN,4,19,8,9,5,3,...,3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,sr:sport_event:43216897,sr:competitor:15126,"Murray, Andy",NaN,2,13,22,6,2,7,...,-5,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0


In [42]:
mens['rs_home'] = mens['seed_home'].fillna(mens['rank_home'])
mens['rs_away'] = mens['seed_away'].fillna(mens['rank_away'])
missing_men = list(pd.unique(mens[mens['rs_home'].isna()].name_home))+list((pd.unique(mens[mens['rs_away'].isna()].name_away)))
pd.unique(missing_men)

/var/folders/7_/mtdhgw6s63dc18509m14sztm0000gn/T/ipykernel_62551/363898201.py:4: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  pd.unique(missing_men)


array(['van Assche, Luca', 'Wolf, Jeffrey John', 'Galan, Daniel Elahi',
       'Barrios Vera, Marcelo Tomas', 'Etcheverry, Tomas Martin',
       'Tirante, Thiago Agustin', 'Olivieri, Genaro Alberto',
       'Varillas, Juan Pablo', 'Arnaldi, Matteo', 'Shevchenko, Alexander',
       'Kotov, Pavel', 'Sweeny, Dane', 'Marozsan, Fabian',
       'Seyboth Wild, Thiago', 'Machac, Tomas', "O'Connell, Christopher",
       'De Jong, Jesper', 'Halys, Quentin', 'Kypson, Patrick',
       'Cobolli, Flavio', 'Carballes Baena, Roberto',
       'Bautista Agut, Roberto', 'Lestienne, Constant', 'Popyrin, Alexei',
       'Duckworth, James', 'Bergs, Zizou', 'Shapovalov, Denis',
       'Altmaier, Daniel', 'Vukic, Aleksandar', 'Tabilo, Alejandro',
       'Zhang, Zhizhen', 'Eubanks, Christopher', 'Murray, Andy',
       'Hanfmann, Yannick', 'Marterer, Maximilian', 'Muller, Alexandre',
       'Kecmanovic, Miomir', 'Jasika, Omar', 'Hijikata, Rinky',
       'Zapata Miralles, Bernabe', 'McCabe, James', 'Safiullin, R

In [1]:
womens[~womens['tiebreaks_won'].isna()]
#womens['tiebreaks_won_home'].combine_first(womens['tiebreaks_won'])
#womens['tiebreaks_won_home']

NameError: name 'womens' is not defined